In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from PIL import Image
import cv2
import time
import torchvision
from tqdm.notebook import tqdm_notebook
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
from torchsummary import summary
import time
from torch.autograd import Variable
import os, shutil
from torch.autograd import Variable
import torchvision.utils as vutils
import matplotlib.animation as animation
from mpl_toolkits.axes_grid1 import ImageGrid
import sys, time

In [ ]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
number_of_gpus = torch.cuda.device_count()
device = torch.device("cuda")
device

In [ ]:
df = pd.read_csv(r"C:\Users\ragha\Documents\NVIDIA Internship\final_data_path.csv")
df.head()

In [ ]:
lms_data = {}
faces_data = {}

for i in range (0, len(df)):
    count = 0
    count1 = 0
    basepath = Path('lms_data' +  df['faces'][i][19:] + '/')
    basepath1 = Path('faces_data' +  df['faces'][i][19:] + '/')
    files_in_basepath = basepath.iterdir()
    for item in files_in_basepath:            
        item = str(item)
        files_in_basepath1 = basepath1.iterdir()    
        for item1 in files_in_basepath1:
            item1 = str(item1)
            faces_data[item] = item1
            break

In [ ]:
final_data = list(faces_data.items())
random.shuffle(final_data)
#train_data = final_data[0: int(0.7999 * len(final_data))]
#val_data = final_data[int(0.7999 * len(final_data)): int(np.round((0.8 * len(final_data)), decimals = 0))]
#test_data = final_data[int(np.round((0.8 * len(final_data)), decimals = 0)): ]
train_data = final_data[ : -25]
val_data = final_data[-25: ]

In [ ]:
train_lms = np.array(train_data)[:, 0]
train_faces = np.array(train_data)[:, 1]
val_lms = np.array(val_data)[:, 0]
val_faces = np.array(val_data)[:, 1]
#test_lms = np.array(test_data)[:, 0]
#test_faces = np.array(test_data)[:, 1]

In [ ]:
print("Total dataset size -", len(final_data))
print("Train size -", len(train_data))
print("Validation size -", len(val_data))
#print("Test size -", len(test_data))

In [ ]:
class Dataset(Dataset):
    def __init__(self):
        self.lms = train_lms
        self.faces = train_faces
        
    def __len__(self):
        return len(self.faces)
    
    def __getitem__(self, idx):
        lms_path = train_lms[idx]
        faces_path = train_faces[idx]
        img1 = np.load(lms_path, allow_pickle=True)
        img1 = cv2.resize(img1, dsize=(64, 1000))
        img1 = (img1-127.5)/127.5
        img2 = cv2.imread(faces_path)[:,:,::-1]
        img2 = (img2-127.5)/127.5
        img1_tensor = torch.from_numpy(img1)
        img2_tensor = torch.from_numpy(img2)
        img1_tensor = img1_tensor.view(64, 1000)
        img2_tensor = img2_tensor.view(3, 128, 128)
        
        return img1_tensor.float(), img2_tensor.float()

In [ ]:
dataset1 = Dataset()
data_loader1 = DataLoader(dataset1, batch_size=64, shuffle=False, pin_memory=True) 

In [ ]:
for imgs, labels in data_loader1:
    print("Batch of images has shape: ",imgs.shape)
    print("Batch of labels has shape: ", labels.shape)
    break

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1=nn.Conv1d(64,64,3,stride=2)
        self.bn1=nn.BatchNorm1d(64)
        self.relu=nn.LeakyReLU()
        self.conv2=nn.Conv1d(64,128,3,stride=2)
        self.bn2=nn.BatchNorm1d(128)
        self.conv3=nn.Conv1d(128,512,3,stride=2)
        self.bn3=nn.BatchNorm1d(512)
        self.conv4=nn.Conv1d(512,256,3,stride=2)
        self.bn4=nn.BatchNorm1d(256)
        self.conv5=nn.Conv1d(256,64,3,stride=2)
        self.bn5=nn.BatchNorm1d(64)
        self.pool1=nn.AvgPool1d(2)
        self.pool2=nn.AvgPool1d(2)
        self.pool3=nn.AvgPool1d(2)
        self.pool4=nn.AvgPool1d(2)
        
        self.deconv1=nn.utils.parametrizations.spectral_norm(nn.ConvTranspose2d(64,512,4,stride = 1))
        self.bn6=nn.BatchNorm2d(512)
        self.deconv2=nn.utils.parametrizations.spectral_norm(nn.ConvTranspose2d(512,256,4,stride=2,padding=1))
        self.bn7=nn.BatchNorm2d(256)
        self.deconv3=nn.utils.parametrizations.spectral_norm(nn.ConvTranspose2d(256,128,4,stride=2,padding=1))
        self.bn8=nn.BatchNorm2d(128)
        self.deconv4=nn.utils.parametrizations.spectral_norm(nn.ConvTranspose2d(128,64,4,stride=2,padding=1))
        self.bn9=nn.BatchNorm2d(64)
        self.deconv5=nn.utils.parametrizations.spectral_norm(nn.ConvTranspose2d(64,16,4,stride=2,padding=1))
        self.bn10=nn.BatchNorm2d(16)
        self.deconv6=nn.utils.parametrizations.spectral_norm(nn.ConvTranspose2d(16,3,4,stride=2,padding=1))
        self.bn11=nn.BatchNorm2d(3)
        self.tanh = nn.Tanh()
        
    def forward(self,x):
        #batch_size=x.size()[0]
        #x = x.view(-1, 64, 1000)
        x=self.relu(self.bn1(self.conv1(x)))
        x=self.relu(self.bn2(self.conv2(x)))
        x=self.relu(self.bn3(self.conv3(x)))
        x=self.relu(self.bn4(self.conv4(x)))
        x=self.relu(self.bn5(self.conv5(x)))
        x=self.pool1(x)
        x=self.pool2(x)
        x=self.pool3(x)
        x=self.pool4(x)
        
        x=x.view(-1,64,1,1)
        
        x=self.relu(self.bn6(self.deconv1(x)))
        x=self.relu(self.bn7(self.deconv2(x)))
        x=self.relu(self.bn8(self.deconv3(x)))
        x=self.relu(self.bn9(self.deconv4(x)))
        x=self.relu(self.bn10(self.deconv5(x)))
        x=self.tanh(self.bn11(self.deconv6(x)))
        
        return x

In [ ]:
gen = Generator().to(device)

if (device.type == 'cuda') and (number_of_gpus > 1):
    gen = nn.DataParallel(gen, list(range(number_of_gpus)))

gen.apply(weights_init)

summary(gen, (64, 1000))

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1=nn.Conv2d(3,16,4)
        self.bn1=nn.BatchNorm2d(16)
        self.relu=nn.LeakyReLU()
        self.conv2=nn.utils.parametrizations.spectral_norm(nn.Conv2d(16,64,3,stride=2))
        self.bn2=nn.BatchNorm2d(64)
        self.conv3=nn.utils.parametrizations.spectral_norm(nn.Conv2d(64,128,3,stride=2))
        self.bn3=nn.BatchNorm2d(128)
        self.conv4=nn.utils.parametrizations.spectral_norm(nn.Conv2d(128,256,3,stride=2))
        self.bn4=nn.BatchNorm2d(256)
        self.conv5=nn.utils.parametrizations.spectral_norm(nn.Conv2d(256,64,3,stride=2))
        self.bn5=nn.BatchNorm2d(64)
        self.flat = nn.Flatten()
        self.fc1=nn.Linear(2304, 1)
        self.sigmoid=nn.Sigmoid()

    def forward(self,x):
        
        x=self.relu(self.bn1(self.conv1(x)))
        x=self.relu(self.bn2(self.conv2(x)))
        x=self.relu(self.bn3(self.conv3(x)))
        x=self.relu(self.bn4(self.conv4(x)))
        x=self.relu(self.bn5(self.conv5(x)))
        x = self.flat(x)
        x=self.sigmoid(self.fc1(x))

        return x

In [ ]:
disc = Discriminator().to(device)

if (device.type == 'cuda') and (number_of_gpus > 1):
    disc = nn.DataParallel(disc, list(range(number_of_gpus)))

disc.apply(weights_init)

summary(disc, (3,128,128))

In [ ]:
criterion=nn.BCELoss().to(device)
gen_optim=torch.optim.Adam(gen.parameters(), lr=0.0002)
disc_optim=torch.optim.Adam(disc.parameters(), lr=0.00015)
scheduler = torch.optim.lr_scheduler.StepLR(disc_optim, step_size = 1, gamma=0.9)

In [ ]:
class Dataset(Dataset):
    def __init__(self):
        self.lms = val_lms
        self.faces = val_faces
        
    def __len__(self):
        return len(self.faces)
    
    def __getitem__(self, idx):
        lms_path = val_lms[idx]
        faces_path = val_faces[idx]
        img1 = np.load(lms_path, allow_pickle=True)
        img1 = cv2.resize(img1, dsize=(64, 1000))
        img1 = (img1-127.5)/127.5
        img2 = cv2.imread(faces_path)[:,:,::-1]
        img2 = (img2-127.5)/127.5
        img1_tensor = torch.from_numpy(img1)
        img2_tensor = torch.from_numpy(img2)
        img1_tensor = img1_tensor.view(64, 1000)
        img2_tensor = img2_tensor.view(3, 128, 128)
        
        return img1_tensor.float(), img2_tensor.float()
    
dataset2 = Dataset()
data_loader2 = DataLoader(dataset2, batch_size=25, shuffle=False, pin_memory=True) 

def val_testing(model, epoch):

    fake_imgs = []
    x = 0
    y = 0
    count = 0
    with torch.no_grad():
    
        for batch_x, batch_y in data_loader2:
            
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            prediction = model(batch_x).detach().cpu()
            prediction = (prediction*127.5)+127.5  
            grid_img = torchvision.utils.make_grid(prediction, nrow=5, normalize = True)
            plt.figure(figsize=(8,8))
            plt.imshow(grid_img.permute(1, 2, 0))
            plt.show()
            #plt.savefig(r'C:/Users/ragha/Documents/NVIDIA Internship/generated_images/generated_img{}.png'.format(epoch), bbox_inches='tight')

In [ ]:
def train():
    img_list = []
    G_losses = []
    D_losses = []
    iters = 0
    fake_label = 0.0 
    num_epochs = 1000
    count = 0
    final_gen_loss = []
    final_disc_loss = []

    for epoch in range(num_epochs):
        start_time = time.time()
        count = count+1
        for batch_x, batch_y in data_loader1:
            
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            
            disc.zero_grad()
            real_data = batch_y.to(device)
            batch_size = real_data.size(0)
            noise = (0.01 - 0.00) * torch.rand(batch_size, 3, 128, 128) + 0.00
            noise = noise.to(device)
            real_data = real_data + noise
            label = (1.11 - 0.9) * torch.rand(batch_size) + 0.9
            label = label.to(device)
            output = disc(real_data).view(-1)
            disc_err_real = criterion(output, label)
            disc_err_real.backward()
            
            noise = (0.01 - 0.00) * torch.rand(batch_size, 64, 1000) + 0.00
            noise = noise.to(device)
            batch_x = batch_x + noise
            fake = gen(batch_x)
            noise = (0.01 - 0.00) * torch.rand(batch_size, 3, 128, 128) + 0.00
            noise = noise.to(device)
            fake = fake + noise
            label.fill_(fake_label)
            label = label.to(device)
            output = disc(fake.detach()).view(-1)
            disc_err_fake = criterion(output, label)
            disc_err_fake.backward()
            disc_err = disc_err_real + disc_err_fake
            disc_optim.step()

            gen.zero_grad()
            label = (1.3 - 0.7) * torch.rand(batch_size) + 0.7
            label = label.to(device)
            output = disc(fake).view(-1)
            gen_err = criterion(output, label)
            gen_err.backward()
            gen_optim.step()

            G_losses.append(gen_err.item())
            D_losses.append(disc_err.item())
                        
            
        final_gen_loss.append(sum(G_losses)/len(G_losses))
        final_disc_loss.append(sum(D_losses)/len(D_losses))                            

        end_time = time.time()
        time_taken = end_time-start_time

        print("Time taken:", time_taken, "seconds")
        print("Epoch: ", epoch+1)
        print('Loss_D: %.4f \tLoss_G: %.4f' % (final_disc_loss[-1], final_gen_loss[-1]))
        val_testing(gen, count)
        
        scheduler.step()

        torch.save({
            'epoch': epoch,
            'gen_state_dict': gen.state_dict(),
            'gen_optimizer_state_dict': gen_optim.state_dict(),
            'gen_loss': G_losses[-1],
            }, 'C:/Users/ragha/Documents/NVIDIA Internship/generator_model.pt')
        
        torch.save({
            'epoch': epoch,
            'disc_state_dict': disc.state_dict(),
            'disc_optimizer_state_dict': disc_optim.state_dict(),
            'disc_loss': D_losses[-1],
            }, 'C:/Users/ragha/Documents/NVIDIA Internship/discriminator_model.pt')
        
        G_losses = []
        D_losses = []
        
    return final_gen_loss, final_disc_loss

        
loss1, loss2 = train()

In [ ]:
plt.plot(loss1)
plt.plot(loss2)
plt.show()

In [ ]:
import wandb
wandb.init()